# What you will learn today:

- How organisms are classified by scientists.
- How to classify the sequences present in gut microbiome samples.

## Taxonomy classification

Biologists classify organisms into groups based on similar characteristics. There is an entire scientific discipline for naming and classifying organisms called Taxonomy. Organisms are classified into a structural hierarchy where each group is contained, within a larger group. The highest level groups are the largest, most general and contain a wide variety of organisms (can be both living or extinct). They are split into even smaller groups, which contain organisms with even more similar features. Each classification group, or level in the hierarchy is called a taxon. The most basic taxon is the species, a group of closely related organisms that can produce viable offspring that can also reproduce.  The scientific name of each species consists of two parts (in Greek or Latin) – genus name, italicized and always capitalized, and species name, lowercase and italicized (<i>Homo sapiens</i>).

The Linnean system, the most common classification system today, has eight levels of taxa - domain, kingdom, phylum, class, order, family, genus, and species. There are three domains - Archaea, Bacteria, and Eukarya. The Archaea and the Bacteria are prokaryotes (single-cell organisms without a nucleus) differing in structural, genetic, and biochemical characteristics. Eukarya contains eukaryotes – organisms with nucleus and membrane-bound organelles. There are six kingdoms: Archaea, Bacteria, Protista, Fungi, Plantae, and Animalia. 

<a href="https://kids.britannica.com/students/article/biological-classification/611149"><img src="images/Taxa.png" width="300" align="center"></a>

## How do we obtain the OTUs?

The sequence of the 16S rRNA (in our case the V4 region) is a popular target for taxonomy and phylogeny studies because of its highly conserved nature (retains similarity throughout evolution). In order to obtain the sample’s taxonomy groups, or Operational Taxonomic Units (OTUs), that you got acquainted with in T2, the <b>SEQUENCES ARE FIRST CLUSTERED</b>, i.e. combined into groups by similar traits. Generally sequences with > 95% match are considered to represent the same genus, while > 97% matches are considered the same species. [1] Here it should be noted that different sequencing machines are prone to different errors, which can be confused for new species and we would like to avoid that. Also, we assume that 1 sequence = 1 species, but in reality one species can have multiple different copies of a gene and an identical amplified region could be shared by multiple species.

<b>SECOND IS COMPARISON TO A REFERENCE DATABASE</b> for taxonomic assignment. In our case, there are many studies that have already attempted to describe the gut microbiome, so we have a reference database. In an open reference type of clustering we cluster similar sequences to the ones that are found in the data base and perform de novo clustering on sequences that are very different. In the least computationally expensive procedure of closed reference clustering we drop those sequences, which would work for thoroughly studied samples such as human stool. In de novo clustering approaches we use the distance between sequences to obtain OTUs rather than the distance to a reference database. Here the groups resulting from clustering can change with addition of more data [2]. This is very computationally expensive. We are going to use closed reference clustering in our study.

In order to find what taxa are present we classify the ASVs that we already have and associate them with taxa names using the q2-feature-classifier plugin. We will use a pre-trained naive Bayes machine-learning classifier that was trained to differentiate taxa present in the 99% Greengenes 13_8 reference set, specifically aimed at the V4 hypervariable region. A pre-trained classifier is a model that was trained on a large benchmark dataset to solve a problem similar to the one that we have to solve. <a href="https://www.youtube.com/watch?v=O2L2Uv9pdDA">Naive bayes</a> assigns labels to sequences, based on prior probability. The classifier works by identifying k-mers (substrings of a sequence) that are diagnostic for particular taxonomic groups, and using that information to predict the taxonomic affiliation of each ASV. We can download the pre-trained classifier:

In [ ]:
%%bash
qiime info
wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2022.2/common/gg-13-8-99-515-806-nb-classifier.qza"

Naive Bayes classifiers perform best when they are trained for the specific hypervariable region that was amplified in the particular study. You can train a classifier specific for your dataset or download pre-trained classifiers for other datasets from the QIIME 2 resource page. Let's classify our representative sequences.

In [ ]:
%%bash
qiime feature-classifier classify-sklearn \
  --i-reads ./rep-seqs.qza \
  --i-classifier ./gg-13-8-99-515-806-nb-classifier.qza \
  --o-classification ./taxonomy.qza

Let’s see the assigned taxonomy associated with the sequences.

In [ ]:
%%bash
qiime metadata tabulate \
  --m-input-file ./taxonomy.qza \
  --o-visualization ./taxonomy.qzv

Let’s also tabulate the representative sequences. This will allow us to see the sequence assigned to the identifier and interactively blast the sequence against the NCBI database.

In [ ]:
%%bash
qiime feature-table tabulate-seqs \
  --i-data ./rep-seqs.qza \
  --o-visualization ./rep-seqs.qzv

Can you answer the following questions? 

Find the feature, 002e78333d6cf2b11aa7a5ba03dd2c68. What is the taxonomic classification of this sequence? What’s the confidence for the assignment? How many features are classified as g__Lactobacillus? Use the tabulated representative sequences to look up these features. If you blast them against NCBI, do you get the same taxonomic identifier as you obtained with q2-feature-classifier?

Let’s look at the taxonomic composition of the samples. To visualize this, we will build a taxonomic barchart of the samples we analyzed in the diversity dataset.

In [ ]:
%%bash
qiime taxa barplot \
  --i-table ./table.qza \
  --i-taxonomy ./taxonomy.qza \
  --m-metadata-file ./sample-metadata.tsv \
  --o-visualization ./taxa_barplot.qzv

Visualize the data at level 2 (phylum level) and sort the samples by sample-type. Can you observe a consistent difference in phylum between stool and swab samples? 

[1]	J. S. Johnson et al., “Evaluation of 16S rRNA gene sequencing for species and strain-level microbiome analysis,” Nat. Commun., vol. 10, no. 1, p. 5029, Nov. 2019, doi: 10.1038/s41467-019-13036-1.

[2]	S. L. Westcott and P. D. Schloss, “De novo clustering methods outperform reference-based methods for assigning 16S rRNA gene sequences to operational taxonomic units,” PeerJ, vol. 3, p. e1487, 2015, doi: 10.7717/peerj.1487.